In [37]:
# Importações necessárias
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Definir semente para reprodutibilidade
tf.random.set_seed(42)
np.random.seed(42)

In [38]:
# --- Parâmetros ---
# A InceptionResNetV2 foi pré-treinada com este tamanho:
IMAGE_SIZE = (299, 299)
BATCH_SIZE = 8  # Use um batch size pequeno devido ao número reduzido de imagens
NUM_CLASSES = 3  # Substitua pelo número real de pessoas/classes que você tem
# --- Caminhos ---
train_dir = 'dataset/treino'
validation_dir = 'dataset/validacao'

In [39]:
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

# Gerador para o conjunto de treinamento com Aumento de Dados
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,       # Rotação de até 20 graus
    width_shift_range=0.2,   # Deslocamento horizontal
    height_shift_range=0.2,  # Deslocamento vertical
    shear_range=0.2,         # Cisalhamento
    zoom_range=0.2,          # Zoom aleatório
    horizontal_flip=True,    # Espelhamento horizontal (comum em faces)
    fill_mode='nearest'
)

# Gerador para o conjunto de validação (somente pré-processamento)
validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Criação dos geradores de fluxo de dados
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.
Found 6 images belonging to 3 classes.


In [40]:
# 3.1. Carregar a InceptionResNetV2 pré-treinada
# include_top=False: Remove a camada de classificação original (1000 classes do ImageNet)
# weights='imagenet': Carrega os pesos pré-treinados
base_model = InceptionResNetV2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
)

# 3.2. Congelar as camadas da base
# Isso impede que os pesos pré-treinados sejam alterados durante o treinamento
for layer in base_model.layers:
    layer.trainable = False

# 3.3. Adicionar as novas camadas de classificação (a "cabeça")
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduz a dimensão para um vetor de características
x = Dense(1024, activation='relu')(x)  # Camada Densa com ReLU
x = Dropout(0.5)(x)  # Dropout para evitar overfitting
predictions = Dense(train_generator.num_classes, activation='softmax')(x) # Camada de Saída

# 3.4. Construir o modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# 3.5. Compilar o modelo
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # Taxa de aprendizado baixa
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Exibir o resumo do modelo (as camadas da base estarão "congeladas")
# model.summary()

In [41]:
# --- Treinamento ---
EPOCHS = 50 # Número pequeno de épocas para começar
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.2857 - loss: 1.0473 - val_accuracy: 0.5000 - val_loss: 1.2257
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 764ms/step - accuracy: 0.2500 - loss: 1.9915 - val_accuracy: 0.5000 - val_loss: 1.1527
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 807ms/step - accuracy: 0.1429 - loss: 1.5939 - val_accuracy: 0.5000 - val_loss: 1.1337
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 786ms/step - accuracy: 0.2500 - loss: 1.4157 - val_accuracy: 0.5000 - val_loss: 1.0657
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 810ms/step - accuracy: 0.7143 - loss: 0.8225 - val_accuracy: 0.5000 - val_loss: 1.0253
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 783ms/step - accuracy: 0.5000 - loss: 1.1093 - val_accuracy: 0.5000 - val_loss: 0.9660
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 819ms/step - accuracy: 0.7143 - loss: 0.8948 - val_accuracy: 0.6667 - val_loss: 0.9359
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 816ms/step - accuracy: 0.3750 - loss: 1.1172 - val_accuracy: 0.6667 - val_loss: 

In [42]:
# 5.1. Descongelar algumas camadas do modelo base
base_model.trainable = True

# Congelar as primeiras N camadas para não destruir o aprendizado inicial
# (ex: manter as primeiras 250 camadas congeladas)
for layer in base_model.layers[:250]:
    layer.trainable = False

# 5.2. Recompilar o modelo com uma taxa de aprendizado muito baixa
# É essencial usar uma taxa de aprendizado baixa para evitar estragar os pesos pré-treinados.
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# model.summary() # Verifique quais camadas estão treináveis agora

# 5.3. Continuar o treinamento (Fine-Tuning)
FINE_TUNE_EPOCHS = 10
total_epochs = EPOCHS + FINE_TUNE_EPOCHS

history_ft = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1], # Continuar de onde parou
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

Epoch 50/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step - accuracy: 1.0000 - loss: 0.2088 - val_accuracy: 0.8333 - val_loss: 0.4725
Epoch 51/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 734ms/step - accuracy: 1.0000 - loss: 0.0943 - val_accuracy: 0.8333 - val_loss: 0.4718
Epoch 52/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 894ms/step - accuracy: 1.0000 - loss: 0.2138 - val_accuracy: 0.8333 - val_loss: 0.4712
Epoch 53/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 752ms/step - accuracy: 1.0000 - loss: 0.1982 - val_accuracy: 0.8333 - val_loss: 0.4705
Epoch 54/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 898ms/step - accuracy: 1.0000 - loss: 0.1773 - val_accuracy: 0.8333 - val_loss: 0.4699
Epoch 55/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 761ms/step - accuracy: 1.0000 - loss: 0.1106 - val_accuracy: 0.8333 - val_loss: 0.4695
Epoch 56/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 971ms/step - accuracy: 0.8750 - loss: 0.2136 - val_accuracy: 0.8333 - val_loss: 0.4690
Epoch 57/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 805ms/step - accuracy: 1.0000 - loss: 0.1808 - val_accuracy: 0.8333 - va

In [43]:
# Salvar o modelo final
model.save('model.keras')
print("Modelo salvo com sucesso!")

Modelo salvo com sucesso!
